In [1]:
!python --version

Python 3.12.0


In [2]:
!conda create -n medical -c cctbx202208 python=3.7 -y
!source /opt/conda/bin/activate medical && conda install -c cctbx202208 python=3.7 -y

Retrieving notices: ...working... done
Solving environment: ...working... done

## Package Plan ##

  environment location: D:\anaconda\envs\medical

  added / updated specs:
    - python=3.7


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.6.15  |       h5b45459_0         188 KB  cctbx202208
    libsqlite-3.39.2           |       h8ffe710_1         637 KB  cctbx202208
    openssl-1.1.1q             |       h8ffe710_0         5.8 MB  cctbx202208
    pip-22.2.2                 |     pyhd8ed1ab_0         1.5 MB  cctbx202208
    python-3.7.12              |h7840368_100_cpython        17.9 MB  cctbx202208
    python_abi-3.7             |          2_cp37m           4 KB  cctbx202208
    setuptools-59.8.0          |   py37h03978a9_1         1.0 MB  cctbx202208
    sqlite-3.39.2              |       h8ffe710_1         660 KB  cctbx202208
    ucrt-10.0.20348.0          |   



==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.11.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.11.0


'source' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!/opt/conda/envs/medical/bin/python3 --version
!echo 'print("Hello, World!")' > test.py
!/opt/conda/envs/medical/bin/python3 test.py

The system cannot find the path specified.
The system cannot find the path specified.


In [ ]:
!sudo rm /opt/conda/bin/python3
!sudo ln -sf /opt/conda/envs/medical/bin/python3 /opt/conda/bin/python3

In [ ]:
!python3 --version

In [ ]:
!pip install -r /kaggle/input/full-req-chex/full_requirements.txt

In [ ]:
import os
import tensorflow as tf
from CNN_encoder import CNN_Encoder
from configs import argHandler
import time
from medical_w2v_wrapper import Medical_W2V_Wrapper
from tokenizer_wrapper import TokenizerWrapper
import matplotlib.pyplot as plt
from utility import get_optimizer, get_enqueuer
import os
import json
from augmenter import augmenter
from gpt2.gpt2_model import TFGPT2LMHeadModel
from test import evaluate_enqueuer
import pandas as pd
from glob import glob
import shutil

In [ ]:


# # Allow memory growth for GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
# for gpu in gpus:
#     tf.config.experimental.set_memory_growth(gpu, True)
print(gpus)


In [ ]:
from configs import argHandler

In [ ]:
print(tf.version)

In [ ]:
#Creates a arghandler class which takes care of initalizing all the variables

FLAGS = argHandler()
FLAGS.setDefaults()
tf.keras.backend.set_learning_phase(1)

In [ ]:
# ALL THE VARIABLES INITIALIZED BY ARGHANDLER
FLAGS.help()

In [ ]:
len(FLAGS.tags)

In [ ]:
FLAGS.tags

# Then we are tokenizing all the words in our captions data set #

# Tokenizer wrapper takes the arguments

### csv Dataset containing image and captions
### Name of the Column containing caption
### Max Length of Sequence - 200
### Vocab_Size - 1001

In [ ]:
tokenizer_wrapper = TokenizerWrapper(FLAGS.all_data_csv, FLAGS.csv_label_columns[0],
                                     FLAGS.max_sequence_length, FLAGS.tokenizer_vocab_size)

In [ ]:
tokenizer_wrapper.get_token_of_word("we")

In [ ]:
FLAGS.train_csv

In [ ]:
train_enqueuer, train_steps = get_enqueuer(FLAGS.train_csv, FLAGS.batch_size, FLAGS, tokenizer_wrapper)
test_enqueuer, test_steps = get_enqueuer(FLAGS.test_csv, 1, FLAGS, tokenizer_wrapper)
batch_test_enqueuer, batch_test_steps = get_enqueuer(FLAGS.test_csv, FLAGS.batch_size, FLAGS, tokenizer_wrapper)

In [ ]:
train_enqueuer.start(workers=FLAGS.generator_workers, max_queue_size=FLAGS.generator_queue_length)

In [ ]:
#loads the 400D embeddings matrix frm gensim

medical_w2v = Medical_W2V_Wrapper()

In [ ]:

#SAVES WORD EMBEDDINGS FOR ALL THE WORDS IN OUR REPORTS DATASET
medical_w2v.save_embeddings(tokenizer_wrapper.get_word_tokens_list(),FLAGS.tags)

In [ ]:
print(tokenizer_wrapper.get_word_tokens_list())

In [ ]:
#GETS WORD EMBEDDINGS FOR ALL THE WORDS IN OUR REPORTS DATASET
embeddings = medical_w2v.get_embeddings_matrix_for_words(tokenizer_wrapper.get_word_tokens_list(),
                                                          FLAGS.tokenizer_vocab_size)

In [ ]:
tags_embeddings = medical_w2v.get_embeddings_matrix_for_tags(FLAGS.tags)

In [ ]:
print(f"Embeddings shape: {embeddings.shape}")

In [ ]:
print(f"Tags Embeddings shape: {tags_embeddings.shape}")

In [ ]:
del medical_w2v

In [ ]:
from CNN_encoder import CNN_Encoder

In [ ]:
encoder = CNN_Encoder('pretrained_visual_model', FLAGS.visual_model_name, FLAGS.visual_model_pop_layers,
                      FLAGS.encoder_layers,
                      FLAGS.tags_threshold, tags_embeddings, FLAGS.finetune_visual_model, len(FLAGS.tags))

In [ ]:
train_generator = train_enqueuer.get()

In [ ]:
t_img,t_target, _ = next(train_generator)
print(type(t_img))

In [ ]:
print(t_img.shape)

In [ ]:
t_v, t_embed = encoder(t_img)

In [ ]:
print(t_embed[0])

In [ ]:
print(t_embed[0].shape)

In [ ]:
from gpt2.gpt2_model import TFGPT2LMHeadModel
decoder = TFGPT2LMHeadModel.from_pretrained('distilgpt2', from_pt=True, resume_download=True)

In [ ]:
optimizer = get_optimizer(FLAGS.optimizer_type, FLAGS.learning_rate)

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [ ]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, tokenizer_wrapper.GPT2_pad_token_id()))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [ ]:
loss_plot = []

In [ ]:
@tf.function
def train_step(images, target, test_mode=False):
    with tf.GradientTape() as tape:
        visual_features, tags_embeddings = encoder(images)
        dec_input = target[:, 0:-1]

        # passing the features through the decoder
        predictions, _ = decoder(dec_input, visual_features=visual_features, tags_embeddings=tags_embeddings, past=None)

        loss = loss_function(target[:, 1:], predictions)
    if not test_mode:
        trainable_variables = encoder.trainable_variables + decoder.trainable_variables
        gradients = tape.gradient(loss, trainable_variables)
        optimizer.apply_gradients(zip(gradients, trainable_variables))
    return loss

In [ ]:
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer=optimizer)

In [ ]:
try:
    os.makedirs(os.path.join(FLAGS.ckpt_path, 'best_ckpt'))
except:
    print("path already exists")

In [ ]:
with open(os.path.join(FLAGS.ckpt_path, 'configs.json'), 'w') as fp:
    json.dump(FLAGS, fp, indent=4)

In [ ]:
ckpt_manager = tf.train.CheckpointManager(ckpt, FLAGS.ckpt_path, max_to_keep=1)

In [ ]:
start_epoch = 0
best_test_avg_score = 0

In [ ]:
def get_avg_score(scores_dict):
    avg_score = 0
    for value in scores_dict.values():
        avg_score += value
    avg_score = avg_score / len(scores_dict)
    return avg_score

In [ ]:
print(ckpt_manager.latest_checkpoint)

In [ ]:
if ckpt_manager.latest_checkpoint and FLAGS.continue_from_last_ckpt:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Restored from checkpoint: {}".format(ckpt_manager.latest_checkpoint))
    try:
        with open(os.path.join(FLAGS.ckpt_path, 'scores.json')) as scores_file:
            scores = json.load(scores_file)
            best_test_avg_score = get_avg_score(scores)
            print(f"best scores: {scores}")
    except:
        print("No previous scores found")

In [ ]:
train_batch_losses_csv = {"step": [], "batch_loss": []}
test_batch_losses_csv = {"step": [], "batch_loss": []}
train_after_batch_losses_csv = {"step": [], "batch_loss": []}
losses_csv = {"epoch": [], "train_loss": [], "train_after_loss": [], "test_loss": []}
time_csv = {"epoch": [], 'time_taken': [], "scores": []}

In [ ]:
def get_overall_loss(enqueuer, steps, batch_losses_csv):
    tf.keras.backend.set_learning_phase(0)

    if not enqueuer.is_running():
        enqueuer.start(workers=FLAGS.generator_workers, max_queue_size=FLAGS.generator_queue_length)
    generator = enqueuer.get()

    batch_losses = []
    total_loss = 0
    step = 0
    for batch in range(steps):
        img, target, _ = next(generator)
        batch_loss = train_step(img, target, True)
        batch_losses_csv['step'].append(step)
        batch_losses_csv['batch_loss'].append(batch_loss.numpy())
        total_loss += batch_loss
        batch_losses.append(batch_loss)
        step += 1
    epoch_loss = total_loss / generator.steps
    enqueuer.stop()
    tf.keras.backend.set_learning_phase(1)

    return epoch_loss, batch_losses

In [ ]:
train_generator = train_enqueuer.get()

In [ ]:
for epoch in range(start_epoch, 1):
    start = time.time()
    total_loss = 0
    times_to_get_batch = 0
    pure_training_time = 0
    step = 0
    # for batch in range(train_steps):
    for batch in range(1):
        t = time.time()
        img, target, _ = next(train_generator)
        print("Time to get batch: {} s ".format(time.time() - t))
        if time.time() - t > 2:
            times_to_get_batch += 1
        step_time = time.time()
        batch_loss = train_step(img, target)
        pure_training_time += time.time() - step_time
        total_loss += batch_loss
        step += 1
        train_batch_losses_csv['step'].append(step)
        train_batch_losses_csv['batch_loss'].append(batch_loss.numpy())

        print("Time to train step: {} s ".format(time.time() - t))

        if batch % 1 == 0 and batch > 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(
                epoch + 1, batch, batch_loss.numpy()))
    # storing the epoch end loss value to plot later'
    total_loss = (total_loss / train_steps).numpy()
    loss_plot.append(total_loss)
    print('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                        total_loss))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    print('Batches that took long: {}'.format(times_to_get_batch))
    if FLAGS.calculate_loss_after_epoch:
        test_epoch_loss, _ = get_overall_loss(batch_test_enqueuer, batch_test_steps, test_batch_losses_csv)
        train_epoch_loss, _ = get_overall_loss(train_enqueuer, train_steps, train_after_batch_losses_csv)
        losses_csv['train_after_loss'].append(train_epoch_loss.numpy())
        losses_csv['test_loss'].append(test_epoch_loss.numpy())
    else:
        losses_csv['train_after_loss'].append('-')
        losses_csv['test_loss'].append('-')
    losses_csv["epoch"].append(epoch + 1)
    losses_csv['train_loss'].append(total_loss)

    pd.DataFrame(losses_csv).to_csv(os.path.join(FLAGS.ckpt_path, 'losses.csv'), index=False)
    pd.DataFrame(train_batch_losses_csv).to_csv(os.path.join(FLAGS.ckpt_path, 'train_batch_losses.csv'), index=False)
    pd.DataFrame(train_after_batch_losses_csv).to_csv(os.path.join(FLAGS.ckpt_path, 'train_after_batch_losses.csv'),
                                                      index=False)
    pd.DataFrame(test_batch_losses_csv).to_csv(os.path.join(FLAGS.ckpt_path, 'test_batch_losses.csv'), index=False)
    ckpt_manager.save()

    plt.plot(loss_plot)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Loss Plot')
    plt.savefig(FLAGS.ckpt_path + "/loss.png")

    if epoch % FLAGS.epochs_to_evaluate == 0 and epoch > 0:
        current_avg_score = 0
        print("Evaluating on test set..")
        train_enqueuer.stop()
        current_scores = evaluate_enqueuer(test_enqueuer, test_steps, FLAGS, encoder, decoder, tokenizer_wrapper)
        time_csv['epoch'].append(epoch + 1)
        time_csv['time_taken'].append(pure_training_time)
        time_csv['scores'].append(current_scores)
        df = pd.DataFrame(time_csv)
        df.to_csv(os.path.join(FLAGS.ckpt_path, 'time.csv'), index=False)
        current_avg_score = get_avg_score(current_scores)
        train_enqueuer.start(workers=FLAGS.generator_workers, max_queue_size=FLAGS.generator_queue_length)
        if best_test_avg_score == 0 or current_avg_score > best_test_avg_score:
            print(f"found a new best model and saving the ckpt")
            shutil.rmtree(os.path.join(FLAGS.ckpt_path, 'best_ckpt'))
            os.mkdir(os.path.join(FLAGS.ckpt_path, 'best_ckpt'))
            for filename in glob(os.path.join(FLAGS.ckpt_path, '*')):
                if os.path.isfile(filename):
                    shutil.copy(filename, os.path.join(FLAGS.ckpt_path, 'best_ckpt'))
            best_test_avg_score = current_avg_score


train_enqueuer.stop()
plt.show()